In [1]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words_nl=set(nltk.corpus.stopwords.words('dutch'))
stop_words_en=set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bertram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bertram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bertram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/bertram/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
import os
import re

texts = []

for file in os.listdir("data"):
    if re.match(r"\d*\.txt", file):
        with open(f"data/{file}") as text:
            texts.append(text.read())

print(len(texts))

768


In [3]:
import spacy

nlp = spacy.load("nl_core_news_sm")

def lemmatizer(texts):
    texts = [text.replace("\n", " ").strip() for text in texts]
    texts = [text.replace(";", " ").strip() for text in texts]
    texts = [text.replace(",", " ").strip() for text in texts]
    docs = nlp.pipe(texts)
    cleaned_lemmas = [[t.lemma_ for t in doc if t.lemma_ not in stop_words_nl 
                       and t.lemma_ not in stop_words_en 
                       and len(t)>3] for doc in docs]

    return cleaned_lemmas

In [29]:
cleaned = lemmatizer(texts[0:100])
print(len(cleaned))
print(cleaned[0])

100
['Publicatie', '2020-04-24Numac', '2020030602', 'FEDERALE', 'OVERHEIDSDIENST', 'BINNENLANDSE', 'ZAKEN', 'APRIL', '2019', 'wijziging', 'koninklijk', 'besluit', 'juli', '1970', 'vaststelling', 'tarief', 'belasting', 'toegevoegde', 'waarde', 'indeling', 'goed', 'dienst', 'tarief', 'bepalen', 'publicatie', 'betreffen', 'Duits', 'vertaling', 'hierna', 'volgen', 'tekst', 'Duits', 'vertaling', 'april', '2019', 'wijziging', 'koninklijk', 'besluit', 'juli', '1970', 'vaststelling', 'tarief', 'belasting', 'toegevoegde', 'waarde', 'indeling', 'goed', 'dienst', 'tarief', 'bepalen', 'publicatie', 'betreffen', 'Belgisch', 'Staatsblad', 'april', '2019).Deze', 'vertaling', 'op_maken', 'centraal', 'dienst', 'Duits', 'vertaling', 'Malmedy', 'FÖDERALER', 'ÖFFENTLICHER', 'DIENST', 'FINANZEN13', 'APRIL', '2019', 'Gesetz', 'Abänderung', 'hinsichtlich', 'bestimmter', 'Veröffentlichungen', 'Königlichen', 'Erlasses', 'Juli', '1970', 'Festlegung', 'Mehrwertsteuersätze', 'Einteilung', 'Güter', 'Dienstleistung

In [30]:
vect = TfidfVectorizer(stop_words=stop_words_nl,max_features=100)

In [31]:
cleaned_joined = [" ".join(words) for words in cleaned]
vectorised = vect.fit_transform(cleaned_joined)
vect.get_feature_names_out()

array(['19', '2019', '2020', '2021', 'aantal', 'activiteit', 'advies',
       'algemeen', 'ander', 'april', 'art', 'artikel', 'bedoelen',
       'bedrag', 'belgisch', 'bepalen', 'bepaling', 'beperken', 'besluit',
       'betreffen', 'betrekking', 'bevoegd', 'bijzonder', 'brussel',
       'brussels', 'coronavirus', 'covid', 'datum', 'december', 'decreet',
       'dienst', 'drie', 'dringend', 'financiën', 'gelet', 'gemeenschap',
       'geval', 'geven', 'gevolg', 'hetzelfde', 'hoofdstedelijk',
       'houden', 'indien', 'instelling', 'inzake', 'jaar', 'januari',
       'juli', 'juni', 'kader', 'koninklijk', 'lijst', 'maart',
       'maatregel', 'maken', 'minister', 'ministerieel', 'moeten',
       'mogelijk', 'nemen', 'november', 'oktober', 'onderneming',
       'openbaar', 'overeenkomstig', 'overwegende', 'pandemie', 'patiënt',
       'periode', 'persoon', 'raad', 'regering', 'september', 'sociaal',
       'state', 'subsidie', 'termijn', 'tijdelijk', 'tijdens',
       'toepassing', 'tus

In [32]:
idf=vect.idf_
dd=dict(zip(vect.get_feature_names_out(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])

artikel videocommunicatie


In [35]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=20,learning_method='online',random_state=42,max_iter=1) 

lda_top=lda_model.fit_transform(vectorised)

In [36]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
persoon vlaams artikel decreet 19 vlaamse twee 2021 woord covid 

Topic 1: 
decreet gelet maart maatregel voorkomen besluit 19 brussels overwegende artikel 

Topic 2: 
bijzonder gemeenschap 2020 juni kader decreet maart vlaams raad minister 

Topic 3: 
gemeenschap koninklijk artikel besluit 2020 maatregel houden uitvoering betreffen minister 

Topic 4: 
onderneming besluit covid 2020 tijdelijk één artikel bedrag regering decreet 

Topic 5: 
art vlaams besluit algemeen 2020 decreet datum volgen oktober vermelden 

Topic 6: 
artikel hoofdstedelijk brussels juni 2020 bedoelen 2021 maart art indien 

Topic 7: 
artikel 2019 lijst april houden wijziging betrekking voorwaarde openbaar 2020 

Topic 8: 
artikel subsidie moeten regering gemeenschap vermelden 2021 koninklijk minister mogelijk 

Topic 9: 
jaar subsidie 2020 dienst bevoegd koninklijk bepalen voorzien maart videocommunicatie 

Topic 10: 
2021 geval art wijzigen 2019 19 zoals artikel moeten twee 

Topic 11: 
artikel bedoele